# Building a QA System with BERT on Wikipedia


In [ ]:
# collapse-hide 

# line 69 of `run_squad.py` script shows why you might need to install 
# tensorboardX if you have an older version of torch
try:
    from torch.utils.tensorboard import SummaryWriter
except ImportError:
    from tensorboardX import SummaryWriter

Conversely, if you're working in Colab, you can run the cell below. 

In [ ]:
!pip install torch  torchvision -f https://download.pytorch.org/whl/torch_stable.html
!pip install transformers
!pip install wikipedia

## Fine-tuning a Transformer model for Question Answering

To train a Transformer for QA with Hugging Face, we'll need
1. to pick a specific model architecture,
2. a QA dataset, and
3. the training script.

With these three things in hand we'll then walk through the fine-tuning process. 

### 1. Pick a Model
HF identifies the following model types for the QA task: 

- BERT
- distilBERT 
- ALBERT
- RoBERTa
- XLNet
- XLM
- FlauBERT


We'll stick with the now-classic BERT model. 


### 2. QA dataset: SQuAD 
One of the most canonical datasets for QA is the Stanford Question Answering Dataset, or SQuAD, which comes in two flavors: SQuAD 1.1 and SQuAD 2.0. 

In [ ]:
# set path with magic
%env DATA_DIR=./data/squad 

# download the data
def download_squad(version=1):
    if version == 1:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
    else:
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
        !wget -P $DATA_DIR https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
            
download_squad(version=2)

env: DATA_DIR=./data/squad
--2020-05-11 21:36:52--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.109.153, 185.199.108.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘./data/squad/train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  14.6MB/s    in 2.8s    

2020-05-11 21:36:55 (14.6 MB/s) - ‘./data/squad/train-v2.0.json’ saved [42123633/42123633]

--2020-05-11 21:36:56--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4370528 (4.2M) [application/j

### 3. Fine-tuning script

We've chosen a model and we've got some data. Time to train!

All the standard models that HF supports have been pre-trained, which means they've all been fed massive unsupervised training sets in order to learn basic language modeling. In order to perform well at specific tasks (like question answering), they must be trained further -- fine-tuned -- on specific datasets and tasks.


HF helpfully provides a script that fine-tunes a Transformer model on one of the SQuAD datasets, called `run_squad.py`. 

In [ ]:
# download the run_squad.py training script
!curl -L -O https://raw.githubusercontent.com/huggingface/transformers/master/examples/question-answering/run_squad.py

This script takes care of all the hard work that goes into fine-tuning a model and, as such, it's pretty complicated. It hosts no fewer than 45 arguments, providing an impressive amount of flexibility and utility for those who do a lot of training. We'll leave the details of this script for another day, and focus instead on the basic command to fine-tune BERT on SQuAD 1.1 or 2.0. 

Below are the most important arguments for the `run_squad.py` fine-tuning script.

In [ ]:
# fine-tuning your own model for QA using HF's `run_squad.py`
# turn flags on and off according to the model you're training

cmd = [
    'python', 
#    '-m torch.distributed.launch --nproc_per_node 2', # use this to perform distributed training over multiple GPUs
    'run_squad.py', 
    
    '--model_type', 'bert',                            # model type (one of the list under "Pick a Model" above)
    
    '--model_name_or_path', 'bert-base-uncased',       # specific model name of the given model type (shown, a list is here: https://huggingface.co/transformers/pretrained_models.html) 
                                                       # on first execution this initiates a download of pre-trained model weights;
                                                       # can also be a local path to a directory with model weights
    
    '--output_dir', './models/bert/bbu_squad2',        # directory for model checkpoints and predictions
    
#    '--overwrite_output_dir',                         # use when adding output to a directory that is non-empty --
                                                       # for instance, when training crashes midway through and you need to restart it
    
    '--do_train',                                      # execute the training method 
    
    '--train_file', '$DATA_DIR/train-v2.0.json',       # provide the training data
    
    '--version_2_with_negative',                       # ** MUST use this flag if training on SQuAD 2.0! DO NOT use if training on SQuAD 1.1
    
    '--do_lower_case',                                 # ** set this flag if using an uncased model; don't use for Cased Models
    
    '--do_eval',                                       # execute the evaluation method on the dev set -- note: 
                                                       # if coupled with --do_train, evaluation runs after fine-tuning 
    
    '--predict_file', '$DATA_DIR/dev-v2.0.json',       # provide evaluation data (dev set)
    
    '--eval_all_checkpoints',                          # evaluate the model on the dev set at each checkpoint
    
    '--per_gpu_eval_batch_size', '12',                 # evaluation batch size for each gpu
    
    '--per_gpu_train_batch_size', '12',                # training batch size for each gpu
    
    '--save_steps', '5000',                            # how often checkpoints (complete model snapshot) are saved 
    
    '--threads', '8',                                  # num of CPU threads to use for converting SQuAD examples to model features
    
    # --- Model and Feature Hyperparameters --- 
    '--num_train_epochs', '3',                         # number of training epochs - usually 2-3 for SQuAD 
    
    '--learning_rate', '3e-5',                         # learning rate for the default optimizer (Adam in this case)
    
    '--max_seq_length', '384',                         # maximum length allowed for the full input sequence 
    
    '--doc_stride', '128'                              # used for long documents that must be chunked into multiple features -- 
                                                       # this "sliding window" controls the amount of stride between chunks
]

Here's what to expect when executing `run_squad.py` for the first time: 

1. Pre-trained model weights for the specified model type (i.e., `bert-base-uncased`) are downloaded.
2. SQuAD training examples are converted into features (takes 15-30 minutes depending on dataset size and number of threads).
3. Training features are saved to a cache file (so that you don't have to do this again *for this model type*).
4. If `--do_train`, training commences for as many epochs as you specify, saving the model weights every `--save_steps` steps until training finishes. These checkpoints are saved in `[--output_dir]/checkpoint-[step number]` subdirectories.
5. The final model weights and peripheral files are saved to `--output_dir`.
6. If `--do_eval`, SQuAD dev examples are converted into features.
7. Dev features are also saved to a cache file.
8. Evaluation commences and outputs a dizzying assortment of performance scores.





#### Training in Colab
Alternatively, you can execute training in the cell as shown below. We note that standard Colab environments only provide 12GB of RAM. Converting the SQuAD dataset to features is memory intensive and may cause the basic Colab environment to fail silently..  

In [ ]:
# hide

# Execute the training from a standard Jupyter Notebook 
from subprocess import PIPE, STDOUT, Popen

# Live output from run_squad.py is through stderr (rather than stdout). 
# The following command runs the process and ports stderr to stdout
p = Popen(cmd,
          stdout=PIPE,
          stderr=STDOUT)

# Default behavior when using bash cells in jupyter is that you won't see the live output in the cell 
# -- you can only see output once the entire process has finished and then you get it all at once. 
# This is less than ideal when training models that can take hours or days of compute time! 

# This command combined with the above allows you to see the live output feed in the notebook, 
# though it's a bit asynchronous.
for line in iter(p.stdout.readline, b''):
    print(">>> " + line.decode().rstrip())

### Training Output

Successful completion of the `run_squad.py` yields a slew of output, which can be found in the `--output_dir` directory specified above. There you'll find...   

Files for the model's tokenizer:
* `tokenizer_config.json`
* `vocab.txt`
* `special_tokens_map.json`

Files for the model itself:
* `pytorch_model.bin`: these are the actual model weights (this file can be several GB for some models)
* `config.json`: details of the model architecture

Binary representation of the command line arguments used to train this model (so you'll never forget which arguments you used!)
* `training_args.bin`

And if you included `--do_eval`, you'll also see these files:
* `predictions_.json`: the official best answer for each example
* `nbest_predictions_.json`: the top n best answers for each example


Providing the path to this directory to `AutoModel` or `AutoModelForQuestionAnswering` will load your fine-tuned model for use.

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Load the fine-tuned model
tokenizer = AutoTokenizer.from_pretrained("./models/bert/bbu_squad2")
model = AutoModelForQuestionAnswering.from_pretrained("./models/bert/bbu_squad2")

# The other way

## Using a pre-fine-tuned model from the Hugging Face repository





 Let's load one of these pre-fine-tuned models.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# executing these commands for the first time initiates a download of the 
# model weights to ~/.cache/torch/transformers/
tokenizer = AutoTokenizer.from_pretrained("deepset/bert-base-cased-squad2") 
model = AutoModelForQuestionAnswering.from_pretrained("deepset/bert-base-cased-squad2")

## Let's try our model!

Whether you fine-tuned your own or used a pre-fine-tuned model, it's time to play with it! There are three steps to QA: 
1. tokenize the input
2. obtain model scores
3. get the answer span



In [ ]:
question = "Who ruled Macedonia"

context = """Macedonia was an ancient kingdom on the periphery of Archaic and Classical Greece, 
and later the dominant state of Hellenistic Greece. The kingdom was founded and initially ruled 
by the Argead dynasty, followed by the Antipatrid and Antigonid dynasties. Home to the ancient 
Macedonians, it originated on the northeastern part of the Greek peninsula. Before the 4th 
century BC, it was a small kingdom outside of the area dominated by the city-states of Athens, 
Sparta and Thebes, and briefly subordinate to Achaemenid Persia."""


# 1. TOKENIZE THE INPUT
# note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for 
# exploration but you cannot feed that into a model. 
inputs = tokenizer.encode_plus(question, context, return_tensors="pt") 

# 2. OBTAIN MODEL SCORES
# the AutoModelForQuestionAnswering class includes a span predictor on top of the model. 
# the model returns answer start and end scores for each word in the text
answer_start_scores, answer_end_scores = model(**inputs)
answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score

# 3. GET THE ANSWER SPAN
# once we have the most likely start and end tokens, we grab all the tokens between them
# and convert tokens back to words!
tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))

'the Argead dynasty'